# Milestone 4: Analysia and Visualisation 

In [ ]:
# Preamble - imports 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sbn 

In [ ]:
finance_df = pd.read_csv("df_after_Task_6.csv") # All should be cleaned now, dates parsed, etc. 
type(finance_df)

# Milestone 4: Analysis and Visualisation 
## Task 1: Current state of loans 
Summarise currently what percentage of the loans are recovered against the investor funding and the total amount funded. 
Visualise the results of the above. 
Visualise what percentage of the total amount would be recovered up to 6 months in the future. 